In [2]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import re


driver = webdriver.Chrome('/home/eunbi/Desktop/Study/크롤링/chromedriver')
driver.implicitly_wait(10)
driver.get('https://www.kurly.com/shop/main/index.php')
driver.implicitly_wait(10)


for i in range(1, 16):
    # 전체 카테고리 클릭
    driver.find_element_by_css_selector(
    '#gnbMenu > div > div.gnb_main > ul > li.menu1 > a').click()
    driver.implicitly_wait(10)
    # 카테고리 클릭
    driver.find_element_by_css_selector(
        f'#gnbMenu > div > div.gnb_sub > div > ul > li:nth-child({i}) > a > span.tit > span').click()
    driver.implicitly_wait(10)
    subcategory_url = driver.current_url
    
    # 서브카테고리 클릭
    for s in range(2, 10):
        try:
            driver.find_element_by_css_selector(
            f'#lnbMenu > div.inner_lnb > ul > li:nth-child({s}) > a').click()
        except:
            break
        time.sleep(1)
        subcategory_name =  driver.find_element_by_css_selector(
            f'#lnbMenu > div.inner_lnb > ul > li:nth-child({s}) > a')
        print(subcategory_name.text)
        
        # 제품 상세 화면 클릭
        for g in range(1, 17):
            driver.find_element_by_css_selector(
            f'#goodsList > div.list_goods > div > ul > li:nth-child({g}) > div > a').click()
            #sectionView > div > div.thumb
            driver.implicitly_wait(20)
            # soup 설정
            link = driver.current_url
            driver.get(link)
            time.sleep(1)
            html = driver.page_source
            time.sleep(1)
            soup = BeautifulSoup(html, 'html.parser')
            time.sleep(1)

            
            # 썸네일
            img1 = soup.select_one('#sectionView > div > div.thumb > img')
            print(f"img1: {img1['src']}")

            
            # 상품명
            name = soup.select_one('#sectionView > div > p.goods_name > strong').string
            print(f'name: {name}')

            # 짧은설명
            summary = soup.select_one('#sectionView > div > p.goods_name > span.short_desc').string
            print(f'summary: {summary}')

            # 가격
            price = soup.select_one(
            '#sectionView > div > p.goods_price > span.position > span.dc > span.dc_price').get_text()
            print(f'price: {price}')

            ## 할인
            try:
                discount_rate = soup.select_one(
                '#sectionView > div > p.goods_price > span.position > span.dc > span.dc_percent').string
                print(f'discount_rate: {discount_rate}')
            except:
                pass
            
            goods_info = soup.select_one('#sectionView > div > div.goods_info')
            unit, amount, package, made_in = '', '', '', ''
            for dl in goods_info:
                try:
                    dt = dl.select_one('dt.tit')
                    dd = dl.select_one('dd.desc')
                    if dt.text == '판매단위':
                        unit = dd.text
                        continue
                    elif dt.text == '중량/용량':
                        amount = dd.text
                        continue
                    elif dt.text == '원산지':
                        made_in = dd.text.strip()
                        continue
                    elif dt.text == '포장타입':
                        for d in dd:
                            package = d.string.strip()
                            break
                        continue
                except:
                    pass
            print(f'unit: {unit}, amount: {amount}, made_in: {made_in}, package: {package}')
            
            # 큰 이미지

            img2 = soup.select_one(
                '#goods-description > div > div.goods_intro > div.pic > img')
            img2 = img2['src']
            img2 = 'https:'+img2
            print(f'img2: {img2}')

            # 상품설명
            description = soup.select_one(
                '#goods-description > div > div.goods_intro > div.context.last > p').string
            print(f'description: {description}')
            
            # 상품이미지
            img3 = soup.select_one(
                '#goods_pi > p > img')
            img3 = img3['src']
            img3 = 'https:'+img3
            print(f'img3: {img3}')
            time.sleep(1)
            
            # options
            try:
                option_select = driver.find_element_by_class_name("txt_select").click()
                option_url = driver.current_url
                driver.get(option_url)
                html = driver.page_source
                time.sleep(2)
                soup = BeautifulSoup(html, 'html.parser')
                time.sleep(2)
                options = soup.select_one('#cartPut > div.cart_option.cart_type2 > div > div.in_option > div.list_goods > div > div > ul')
                for li in options:
                    for span in li.a:
                        option_name = span.text.strip()
                        break
                    option_price = li.find("span", class_="dc_price").text.strip()
                    print(f'option_name: {option_name}')
                    print(f'option_price: {option_price}')
            except:
                pass    
            print('----------------------------------------------------------------------')
            # 서브카테고리 메뉴로 가기
            driver.get(subcategory_url) 
            time.sleep(1)


기본채소
img1: https://res.kurly.com/mobile/service/goodsview/1910/bg_375x482.png
name: 무농약 통셀러리 500g
summary: 아삭아삭 청량한 식감
price: 3,900원
unit: 1봉, amount: 500g, made_in: 국산, package: 냉장/종이포장
img2: https://img-cf.kurly.com/shop/data/goodsview/20191126/gv40000069747_1.jpg
description: 셀러리는 식이섬유와 수분이 풍부하죠. 특유의 향과 아삭아삭한 식감은 청량함마저 안겨주는데요. 씻어서 그대로 먹기도 하는 채소인 만큼 농약을 걱정하는 분들이 많습니다. 그래서 컬리는 농약을 사용하지 않고 안전하게 기른 무농약 셀러리를 준비했습니다. 통셀러리를 반으로 잘라서 포장했어요. 줄기 중에서도 잎사귀와 가까운 부분은 덜 단단하니 세척한 다음 생으로 마요네즈에 찍어서 드시거나 샐러드로 만들어 보세요. 줄기 아래쪽은 특유의 씁쓸함이 줄어드니 볶음이나 찜 요리에 다양하게 활용할 수 있어요.  잎 부분은 다른 과일이나 채소와 함께 착즙 주스로 만들어 보세요.
img3: https://img-cf.kurly.com/shop/data/goodsview/20200106/gv10000076720_1.jpg
----------------------------------------------------------------------


KeyboardInterrupt: 

In [ ]:
background-image: url("https://img-cf.kurly.com/shop/data/goods/1585704958503y0.jpg");
    #sectionView > div > div.thumb